### Type Hinting

As we all well know, Python uses dynamic typing.

This means that, unlike other statically typed languages such as Java, variables in Python do not have a static type - they are simply labels that reference objects - those objects definitely have a specific type, but the label is nothing more than a "pointer" to some object.

Over time, though, people have found a need to still "assign" a type to variables, especially to function arguments and return types.

And so type hints happened. 

First with external libraries such as the static type checker [mypy](https://mypy.readthedocs.io/en/stable/index.html), then later integrated directly into Python itself.

The thing with type hints, is that they in no way modify how Python **runs**. They do get compiled along with the rest of your code, but they do not affect how Python executes your code. 

Let's take a look at an example:

In [1]:
def mult(a: int, b: int):
    return a * b

According to the type hints, `a` and `b` should be integers, and we can call the function with integers:

In [2]:
mult(3, 5)

15

But, we are entirely free to pass other types to the same function, without raising any exceptions:

In [3]:
mult('a', 10)

'aaaaaaaaaa'

Since multiplication between strings and integers is well-defined in Python, the function executes normally.

As you can see, the type hints *do not* affect how Python runs.

So, why use type hinting at all?

There are a number of positives about using type hints, even if Python basically ignores them.

- helps IDEs display context sensitive help info (such as not only the function parameters, but also what the expected type is)
- can help document your code, and in fact several automated document generators are able to use those type hints when they generate documentation
- 3rd party tools can be used to perform static type checking (basically it can help identify bugs, where you pass an unexpected type for a callable)

Some libraries in Python, including some included with the standard library can also leverage those type hints.

For example, the Data Classes code generator can use type hints to generate your classes.

3rd party libraries can also leverage it - for example [pydantic](https://pydantic-docs.helpmanual.io/) uses it for run-time type validation, as well as additional things such as serialization and deserialization.

The API framework [FastAPI](https://fastapi.tiangolo.com/) uses Pydantic, and this allows not only a very simple way of validating inputs, but also formatting outputs, as well as auto generating API swagger documentation.

However, it's not all positives - the addition of type hinting does make the code a little more verbose, and in some cases a little less readable, as well as taking more time to put the type hints in. 

There may be a slight startup time overhead, especially if you use the `typing` module, since that module has to be loaded up.

But in general, the benefits of type hinting outweight any negatives.

So let's look at how we can specify some type hints.

For this video, we'll stick to relatively simple cases, but as hinting continues to evolve, more and more becomes possible.

For standard types, hinting is very easy, and we saw an example earlier.

Let's look at another one:

In [4]:
def func(a: dict, b: list, c: bool = True):
    pass

Here you can see that we are able to use type hints as well as defaults at the same time, and it works just fine with keyword-only arguments as well:

In [5]:
def func(*, a: int, b: int = 0):
    pass

We can also specify the return type hint:

In [6]:
def func(a: int, b: int) -> str:
    return f"{a=}, {b=}"

Or, for a function that does not have a return value, remembering that functions **always** return something, and in the absence of an explicit `return` statement, will return `None`.

In [7]:
def func(a: str) -> None:
    print(a)

Where things get more interesting, is when dealing with more complex types, or when we are willing to accept different types for the same argument.

Let's start with type hinting a function that can accept either an `int` or a `float` for some parameter.

To do this, we'll need to use the `typing` module.

To define a type hint that can accept one of multiple types, we can use `Union` from typing:

In [8]:
from typing import Union

In [9]:
def mult(a: Union[str, int], b: int) -> Union[str, int]:
    return a * b

Here we are saying that `a` can be either a string or an integer, `b` should be an integer, and the result will be either a string or an integer.

In more recent versions of Python (3.10+), you can skip the use of `Union` and use the `|` operator instead:

In [10]:
def mult(a: str | int, b: int) -> str | int:
    return a * b

We can also use the `Any` annotation to indicate that any type is acceptable:

In [11]:
from typing import Any

In [12]:
def fmt(a: Any) -> None:
    print(str(a))

We can also indicate that an argument can be `None` by using the `Optional` annotation - this is not the same thing as an optional parameter, this is more for specifying that an argument is required, but can be either some specific type (or types), or even `None`:

In [13]:
from typing import Optional

In [14]:
def func(a: Optional[int]) -> None:
    pass

And using the `|` notation, we can also write it this way:

In [15]:
def func(a: int | None) -> None:
    pass

So far, we've still been using simple types, but we can also annotate container types, such as lists, tuples, dictionaries and so on.

We could definitely indicate an argument is a list this way:

In [16]:
def func(l: list) -> None:
    pass

But we can go further and specify that the list should be a list of floats for example.

To do this, we need to use something called generic types (or simply generics).

Let's try the list annotation first:

In [17]:
from typing import List

In [18]:
def func(l: List[float]) -> List[int]:
    return [int(el) for el in l]

Again, just to reiterate, Python will gladly accept a list of integers here, or even a list of strings - of course you may run into a run-time exception if you do, but Python is not going to complain otherwise:

In [19]:
func(['1', '2', '3'])

[1, 2, 3]

There are plenty of generics available in the `typing` module, so you should read up about them in the Python docs. (and as I'll mention later, some of these are generics in the `typing` module are being deprecated, and instead we can use the ones becoming available in other modules). For now, especially if you want to be backward compatible, we'll use the ones in the `typing` module.)

So let's look at a few more.

Sometimes, we want to indicate that a sequence of objects can be a list or a tuple, or really any **sequence** type containing elemnts of a certain type.

We can do this using the more general `Sequence` generic:

In [20]:
from typing import Sequence

In [21]:
def square(numbers: Sequence[float | int]) -> List[float | int]:
    return [el * el for el in numbers]

In cases where we may be re-using the same annotation over and over again, maybe we're building a module of functions that mostly take a sequence of floats or integers as we saw in the function above.

Re-writing the same hint again and again, can become tedious, so we can actually define the type hint annotation itself as a variable:

In [22]:
Vector = Sequence[float | int]

In [23]:
def norm(v: Vector) -> float:
    pass

Let's switch to specifying hints for dictionaries, that are little more specific than just using `dict`.

In [24]:
from typing import Dict

With the `Dict` generic, we can specify type hints for both the keys and the values:

In [25]:
Dict[str, int]

typing.Dict[str, int]

This indicates a dictionary with keys as strings, and values as integers.

This can be combined with other annotations we saw earlier, for example:

In [26]:
Dict[str, int | Sequence[int]]

typing.Dict[str, typing.Union[int, typing.Sequence[int]]]

Or, if dealing with earlier versions of Python:

In [27]:
Dict[str, Union[int, Sequence[int]]]

typing.Dict[str, typing.Union[int, typing.Sequence[int]]]

One last generic I'd like to cover, is how to specify type hints when a parameter is a callable.

Let's look at this function, that replicates the `map` function:

In [28]:
def custom_map(func, sequence):
    for el in sequence:
        yield str(func(el))

So, here we have a few things going on:
- `func` is a function with a single parameter (of undetermined type) that returns an undefined type
- `sequence` needs to be a sequence of elements (of undetermined type)
- `custom_map` is a generator, so it yields results one by one.
- each result yielded by `custom_map` is a string

Let's first fill in the hints we already know how to do:

In [29]:
def custom_map(func, sequence: Sequence[Any]):
    for el in sequence:
        yield str(func(el))

Next, we need to specify that `func` is a function of one variable (of any type).

To do this, we're going to use the `Callable` generic:

In [30]:
from typing import Callable

We can specify both the arguments of the callable, as well as the return type as follows:

In [31]:
Callable[[Any], Any]

typing.Callable[[typing.Any], typing.Any]

So now, we can add this hint to our function:

In [32]:
def custom_map(
    func: Callable[[Any], Any], 
    sequence: Sequence[Any]
):
    for el in sequence:
        yield str(func(el))

Finally, we need to indicate that `custom_map` is generator that yields strings.

For that, we could use the `Iterator` generic:

In [33]:
from typing import Iterator

In [34]:
Iterator[str]

typing.Iterator[str]

And we can now fully annotate our function:

In [35]:
def custom_map(
    func: Callable[[Any], Any], 
    sequence: Sequence[Any]
) -> Iterator[str]:
    for el in sequence:
        yield str(func(el))

Note that for generators, more can be happening than just yielding a result - generators can also receive inputs, as well as return something when they terminate. That can also be annotated, using the `Generator` generic - you can read up more about it [here](https://docs.python.org/3/library/typing.html#typing.Generator)

Let's annotate a callable with a bit more detail, just to get a little bit more practice.

In [36]:
def apply(func, values):
    for value_1, value_2 in values:
        yield func(value_1, value_2)

So, for this function, we expect `func` to be a function of two arguments - let's say we want them to be floats or ints.

We then also expect that values will be an iterable (not necessarily a sequence type) of iterables containing two values (int or float).

For example we can call it this way:

In [37]:
list(apply(lambda x, y: x + y, [(1, 1), (2, 2), (3, 4), [5, 6]]))

[2, 4, 7, 11]

Let's annotate our function:

In [38]:
from typing import Iterable

In [39]:
def apply(
    func: Callable[[int | float, int| float], int | float],
    values: Iterable[Iterable[int | float]]
) -> Iterator[int | float]:
    for value_1, value_2 in values:
        yield func(value_1, value_2)

Type hinting in Python is still evolving, and here we have been using generics such as `Iterable` and `Iterator` that we imported from the `typing` module. But these have since been deprecated, and we really should get them from the `collections.abc` module:

In [40]:
from collections.abc import Iterable, Iterator

They work the same way, it's just that these abstract base classes now support the `[]` we use for type hints, and will therefore eventually be removed from the typing module.

For a full list of these deprecated generics that can now be found in the abstract base classes, see [here](https://docs.python.org/3/library/typing.html#abstract-base-classes)

Another thing to mention is that type hints can be used to annotate any variable, not just function arguments.

In [41]:
class Test:
    a: int = 10
    b: str = 'abc'
    l: List[int | float] = [1, 3.14]

What we've covered here is just a small sample of what's available in type hints, refer to the [python docs](https://docs.python.org/3/library/typing.html#module-typing) on type hints for more info.

In an upcoming video we'll take a look at the Pydantic 3rd party library, and see how it leverages type hints to do some really cool stuff!